<a href="https://colab.research.google.com/github/LCaravaggio/AnalisisCuantitativoAvanzado/blob/main/Ejercicio_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 5

Series de tiempo. Trabajo con datos del EMAE obtenidos con la API de Alphacast.

Primero importamos todas las librerías necesarias

In [ ]:
from google.colab import drive
import json

import requests
from requests.auth import HTTPBasicAuth

import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

Por una cuestión de seguridad, tenemos la clave de la API guardada en el drive. 

Para obtener su API Key de Alphacast tienen que generar un usuario y acceder a https://www.alphacast.io/settings

Después pueden dejar guardada su key en el Drive, o de alguna otra manera. 

Alternativamente podrían bajar el DataSet y cargarlo a mano. También es posible acceder a estos datos directamente desde [INDEC](https://www.indec.gob.ar/indec/web/Nivel4-Tema-3-9-48), pero esos datos no están estandarizados. 

In [ ]:
drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/alphacast.json", 'r') as f:
    api_token= json.load(f)

Mounted at /content/drive


In [ ]:
# Definir user key para la API
user_key = api_token["key"]

Traemos los datos que necesitamos

In [ ]:
# EMAE
emae = requests.get('https://api.alphacast.io/datasets/5331/data?apiKey='+user_key+'&&$format=csv', auth=HTTPBasicAuth(user_key, ''))
open('emae.csv','wb').write(emae.content)
emae = pd.read_csv('emae.csv',delimiter = ',')

In [ ]:
emae['Date'] = pd.to_datetime(emae['Date'])
emae.set_index('Date', inplace = True)
emae = emae.loc['2004-01-01':]

Preparamos una función para graficar

In [ ]:
sns.set_theme(style='darkgrid')
def graficar(variable):
    fig = sns.lineplot(x=emae.index, y=variable)
    plt.title(variable.name)
    plt.xlabel("Año")
    plt.savefig('%s.png'%variable.name)
    return fig

Y la graficamos. Si bien la serie se estancadesde 2011, claramente su media depende del tiempo, por lo que no podemos quedarnos tranquilos que sea estacionaria. Por otro lado se observa su comportamiento estacional ya que lo que trajimos fue la serie original y no la desestacionalizada. 

# Preguntas

Pregunta 1: 
Que la serie sea estacionaria quiere decir que su media y su varianza no dependen del tiempo. ¿Es estacionaria esta serie? 

Ayuda: Tal vez con graficarla alcance.



```
graficar(emae['Emae'])
```

También podría graficar algunas transformaciones de la serie. Por ejemplo su media movil.

```
graficar(emae['Emae'].rolling(window=12).mean())
```

O su desvío estándar móvil



```
graficar(emae['Emae'].rolling(window=12).std())
```





Pregunta 2: Las funciones de autocorrelación (FAC) y autocorrelación parcial (FAP) muestran el grado de vinculación del valor actual respecto de los valores pasados. Graficar. ¿Por qué sobresalen algunos elementos de la FAP?


Ayuda: 

```
from statsmodels.graphics.tsaplots import plot_acf

plot_acf(emae['Emae'], lags=30)
plt.show()
```

Más ayuda: 

```
from statsmodels.graphics.tsaplots import plot_pacf

plot_pacf(emae['Emae'], lags=30)
plt.show()
```






Pregunta 3: ¿Qué pasó en 2008 y 2020?


Ayuda: 



```
from statsmodels.tsa.seasonal import seasonal_decompose

plt.rcParams["figure.figsize"] = (10,10)
decomposition=seasonal_decompose(emae)

original=decomposition.observed
trend=decomposition.trend
seasonal=decomposition.seasonal
residual=decomposition.resid

plt.subplot(411)
plt.plot(original, label='Original')
plt.legend(loc='upper left')
plt.subplot(412)
plt.plot(trend, label='Tendencia')
plt.legend(loc='upper left')
plt.subplot(413)
plt.plot(seasonal, label='Estacional')
plt.legend(loc='upper left')
plt.subplot(414)
plt.plot(residual, label='Residuo')
plt.legend(loc='upper left')
plt.tight_layout()

plt.show()
```



Pregunta 4: ¿Qué son las otras columnas de nuestro Data Set? 

Ayuda: 



```
emae.columns
```

Más ayuda: [EMAE - INDEC](https://www.indec.gob.ar/indec/web/Nivel4-Tema-3-9-48)

Pregunta 5: ¿Tendría sentido armar un modelo que busque explicar el EMAE a partir de las otras columnas del Data Set? 

Ayuda: No.